# Save predictions to Pandas

This notebook loads all 2x5 trained models, gets the predictions and saves the results to pandas dataframe and a file

## Setup
- Download and unpack training data
- Create DonkeyCar project and load configuration

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('donkeycar')
sys.path.append('src')

In [ ]:
import os
# do we have the correct env loaded?
assert 'donkey' == os.environ['CONDA_DEFAULT_ENV'], os.environ['CONDA_DEFAULT_ENV']

In [2]:
![ ! -d "data/" ] && \
mkdir data && \
curl -L "https://www.dropbox.com/s/eltelvjt12a4ny1/data.zip?dl=1" --output data/data.zip && \
unzip data/data.zip -d data

In [23]:
![ ! -d "models/" ] && \
mkdir models && \
curl -L "https://www.dropbox.com/s/k9amvpqz3lcu7f3/models.zip?dl=1" --output models/models.zip && \
unzip models/models.zip -d models

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   134    0   134    0     0    320      0 --:--:-- --:--:-- --:--:--   319
100   342  100   342    0     0    406      0 --:--:-- --:--:-- --:--:--   406
100 58.6M  100 58.6M    0     0  3725k      0  0:00:16  0:00:16 --:--:-- 4233k  0  3684k      0  0:00:16  0:00:14  0:00:02 3996k
Archive:  models/models.zip
  inflating: models/Keras3D_CNN_ModifiedOnlySteering-1.fold-of-5-CC.h5  
  inflating: models/Keras3D_CNN_ModifiedOnlySteering-1.fold-of-5-CW.h5  
  inflating: models/Keras3D_CNN_ModifiedOnlySteering-2.fold-of-5-CC.h5  
  inflating: models/Keras3D_CNN_ModifiedOnlySteering-2.fold-of-5-CW.h5  
  inflating: models/Keras3D_CNN_ModifiedOnlySteering-3.fold-of-5-CC.h5  
  inflating: models/Keras3D_CNN_ModifiedOnlySteering-3.fold-of-5-CW.h5  
  inflating: models/Keras3D_CNN_ModifiedOnlySteering-4.fold-of-5-CC.h5  
  inflating: mod

In [3]:
![ ! -d "mycar/" ] && \
donkey createcar --path mycar && \
cp config/myconfig.py mycar/


### Common settings for both directions

In [3]:
from dataloader import load_data
import numpy as np
import donkeycar as dk
from donkeycar.parts import keras
cfg = dk.load_config(config_path='mycar/config.py')

# None means all the data is used.
# use a smaller size, like 200 for testing end to end.
TUBRECORD_SIZE = None

SEQUENCE_LENGTH = 3
cfg.SEQUENCE_LENGTH = SEQUENCE_LENGTH
cfg.WANDB_ENABLED = False
cfg.TRANSFORMATIONS = ['CROP']
cfg.ROI_CROP_TOP = 60
N_FOLDS = 5
DATA_PATH = 'data'
MODELS_PATH = 'models'

MODEL_CLASS = keras.Keras3D_CNN_ModifiedOnlySteering


________             ______                   _________              
___  __ \_______________  /___________  __    __  ____/_____ ________
__  / / /  __ \_  __ \_  //_/  _ \_  / / /    _  /    _  __ `/_  ___/
_  /_/ // /_/ /  / / /  ,<  /  __/  /_/ /     / /___  / /_/ /_  /    
/_____/ \____//_/ /_//_/|_| \___/_\__, /      \____/  \__,_/ /_/     
                                 /____/                              

using donkey v4.3.5 ...



----------------------------------------------------------------------
Ran 0 tests in 0.000s

OK
INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


loading config file: mycar/config.py
loading personal config over-rides from myconfig.py



### Load Counter Clockwise tubs and get predictions

In [4]:
from dataloader import get_folds
import itertools

DIRECTION = 'CC'

tub_records_80_speed, tub_records_85_speed, tub_records_90_speed = load_data(cfg, DATA_PATH, DIRECTION, TUBRECORD_SIZE)

all_90_speed_data = list(itertools.chain(*list(tub_records_90_speed.values())))
train_folds, test_folds = get_folds(all_90_speed_data)

Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/1-1-CC-80/catalog_4.catalog


INFO:donkeycar.pipeline.types:Loading tubs from paths ['data/1-1-CC-80']
INFO:donkeycar.pipeline.types:Loading tubs from paths ['data/4-1-CC-80']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/4-1-CC-80/catalog_5.catalog


INFO:donkeycar.pipeline.types:Loading tubs from paths ['data/1-3-CC-85']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/1-3-CC-85/catalog_4.catalog


...

Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/4-3-CC-85/catalog_4.catalog


INFO:donkeycar.pipeline.types:Loading tubs from paths ['data/1-2-CC-90']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/1-2-CC-90/catalog_4.catalog


INFO:donkeycar.pipeline.types:Loading tubs from paths ['data/4-2-CC-90']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/4-2-CC-90/catalog_7.catalog


In [5]:
from losses import ModelResults

CC_ModelResults = ModelResults(
    MODEL_CLASS,
    MODELS_PATH,
    SEQUENCE_LENGTH,
    DIRECTION,
    N_FOLDS,
    tub_records_80_speed,
    tub_records_85_speed,
    tub_records_90_speed,
    train_folds,
    test_folds,
    cfg
)

CC_ModelResults.predict_results()


INFO:donkeycar.parts.keras:input_shape (120, 160, 3) num_outputs 1
2022-03-18 18:30:06.628775: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-03-18 18:30:06.670081: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fe0e0320c60 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-03-18 18:30:06.670094: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model models/Keras3D_CNN_ModifiedOnlySteering-1.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model models/Keras3D_CNN_ModifiedOnlySteering-1.fold-of-5-CC.h5


Getting 90-speed  mse-s for fold 1
seq size 3
len ground_truth 4542
36/36 [==============================] - 35s 960ms/step
shape test_preds (4542,)
1-2-CC-90 has 4542 predictions with an mse of 0.09193157655553452
seq size 3
len ground_truth 6229
49/49 [==============================] - 49s 1s/step
shape test_preds (6229,)
4-2-CC-90 has 6229 predictions with an mse of 0.08998082641733327
seq size 3
len ground_truth 2155
17/17 [==============================] - 7s 393ms/step
shape test_preds (2155,)
test90 has 2155 predictions with an mse of 0.09838406395507836
seq size 3
len ground_truth 8616
68/68 [==============================] - 23s 344ms/step
shape test_preds (8616,)
train90 has 8616 predictions with an mse of 0.08890739690188754
Getting 85-speed mse-s for fold 1
seq size 3
len ground_truth 4272
34/34 [==============================] - 33s 959ms/step
shape test_preds (4272,)
1-3-CC-85 has 4272 predictions with an mse of 0.11217945479009678
seq size 3
len ground_truth 4398
35/35 [

INFO:donkeycar.parts.keras:input_shape (120, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model models/Keras3D_CNN_ModifiedOnlySteering-2.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model models/Keras3D_CNN_ModifiedOnlySteering-2.fold-of-5-CC.h5


shape test_preds (4705,)
4-1-CC-80 has 4705 predictions with an mse of 0.09049988554403156
Getting 90-speed  mse-s for fold 2
seq size 3
len ground_truth 4542
36/36 [==============================] - 12s 334ms/step
shape test_preds (4542,)
1-2-CC-90 has 4542 predictions with an mse of 0.10161935555004332
seq size 3
len ground_truth 6229
49/49 [==============================] - 18s 370ms/step
shape test_preds (6229,)
4-2-CC-90 has 6229 predictions with an mse of 0.09525758730023001
seq size 3
len ground_truth 2154
17/17 [==============================] - 6s 368ms/step
shape test_preds (2154,)
test90 has 2154 predictions with an mse of 0.10428143069054202
seq size 3
len ground_truth 8617
68/68 [==============================] - 24s 352ms/step
shape test_preds (8617,)
train90 has 8617 predictions with an mse of 0.09635516087038874
Getting 85-speed mse-s for fold 2
seq size 3
len ground_truth 4272
34/34 [==============================] - 11s 322ms/step
shape test_preds (4272,)
1-3-CC-85 ha

INFO:donkeycar.parts.keras:input_shape (120, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model models/Keras3D_CNN_ModifiedOnlySteering-3.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model models/Keras3D_CNN_ModifiedOnlySteering-3.fold-of-5-CC.h5


shape test_preds (4705,)
4-1-CC-80 has 4705 predictions with an mse of 0.08351193769825634
Getting 90-speed  mse-s for fold 3
seq size 3
len ground_truth 4542
36/36 [==============================] - 12s 326ms/step
shape test_preds (4542,)
1-2-CC-90 has 4542 predictions with an mse of 0.10664093005278963
seq size 3
len ground_truth 6229
49/49 [==============================] - 16s 323ms/step
shape test_preds (6229,)
4-2-CC-90 has 6229 predictions with an mse of 0.11052991788525096
seq size 3
len ground_truth 2154
17/17 [==============================] - 5s 324ms/step
shape test_preds (2154,)
test90 has 2154 predictions with an mse of 0.13408497146221002
seq size 3
len ground_truth 8617
68/68 [==============================] - 22s 330ms/step
shape test_preds (8617,)
train90 has 8617 predictions with an mse of 0.10259196186370258
Getting 85-speed mse-s for fold 3
seq size 3
len ground_truth 4272
34/34 [==============================] - 11s 325ms/step
shape test_preds (4272,)
1-3-CC-85 ha

INFO:donkeycar.parts.keras:input_shape (120, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model models/Keras3D_CNN_ModifiedOnlySteering-4.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model models/Keras3D_CNN_ModifiedOnlySteering-4.fold-of-5-CC.h5


shape test_preds (4705,)
4-1-CC-80 has 4705 predictions with an mse of 0.10449264631444292
Getting 90-speed  mse-s for fold 4
seq size 3
len ground_truth 4542
36/36 [==============================] - 12s 340ms/step
shape test_preds (4542,)
1-2-CC-90 has 4542 predictions with an mse of 0.09716766004397397
seq size 3
len ground_truth 6229
49/49 [==============================] - 16s 329ms/step
shape test_preds (6229,)
4-2-CC-90 has 6229 predictions with an mse of 0.10208405303705559
seq size 3
len ground_truth 2154
17/17 [==============================] - 5s 321ms/step
shape test_preds (2154,)
test90 has 2154 predictions with an mse of 0.11496367041230494
seq size 3
len ground_truth 8617
68/68 [==============================] - 23s 332ms/step
shape test_preds (8617,)
train90 has 8617 predictions with an mse of 0.09627310327931689
Getting 85-speed mse-s for fold 4
seq size 3
len ground_truth 4272
34/34 [==============================] - 11s 326ms/step
shape test_preds (4272,)
1-3-CC-85 ha

INFO:donkeycar.parts.keras:input_shape (120, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model models/Keras3D_CNN_ModifiedOnlySteering-5.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model models/Keras3D_CNN_ModifiedOnlySteering-5.fold-of-5-CC.h5


shape test_preds (4705,)
4-1-CC-80 has 4705 predictions with an mse of 0.08344175398060277
Getting 90-speed  mse-s for fold 5
seq size 3
len ground_truth 4542
36/36 [==============================] - 12s 327ms/step
shape test_preds (4542,)
1-2-CC-90 has 4542 predictions with an mse of 0.10564469531167227
seq size 3
len ground_truth 6229
49/49 [==============================] - 16s 329ms/step
shape test_preds (6229,)
4-2-CC-90 has 6229 predictions with an mse of 0.10764562890663669
seq size 3
len ground_truth 2154
17/17 [==============================] - 5s 313ms/step
shape test_preds (2154,)
test90 has 2154 predictions with an mse of 0.10307195813037912
seq size 3
len ground_truth 8617
68/68 [==============================] - 23s 338ms/step
shape test_preds (8617,)
train90 has 8617 predictions with an mse of 0.10773422676384667
Getting 85-speed mse-s for fold 5
seq size 3
len ground_truth 4272
34/34 [==============================] - 12s 350ms/step
shape test_preds (4272,)
1-3-CC-85 ha

In [10]:
DIRECTION = 'CW'

tub_records_80_speed, tub_records_85_speed, tub_records_90_speed = load_data(cfg, DATA_PATH, DIRECTION, TUBRECORD_SIZE)

all_90_speed_data = list(itertools.chain(*list(tub_records_90_speed.values())))
train_folds, test_folds = get_folds(all_90_speed_data)

CW_ModelResults = ModelResults(
    MODEL_CLASS,
    MODELS_PATH,
    SEQUENCE_LENGTH,
    DIRECTION,
    N_FOLDS,
    tub_records_80_speed,
    tub_records_85_speed,
    tub_records_90_speed,
    train_folds,
    test_folds,
    cfg
)

CW_ModelResults.predict_results()


INFO:donkeycar.pipeline.types:Loading tubs from paths ['data/2-1-CW-80']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/2-1-CW-80/catalog_4.catalog


INFO:donkeycar.pipeline.types:Loading tubs from paths ['data/2-3-CW-85']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/2-3-CW-85/catalog_6.catalog


INFO:donkeycar.pipeline.types:Loading tubs from paths ['data/3-3-CW-85']


...

shape test_preds (4431,)
2-1-CW-80 has 4431 predictions with an mse of 0.05848933363010838
Getting 90-speed  mse-s for fold 3
seq size 3
len ground_truth 1235
10/10 [==============================] - 3s 302ms/step
shape test_preds (1235,)
2-2-CW-90 has 1235 predictions with an mse of 0.07012342325186875
seq size 3
len ground_truth 5217
41/41 [==============================] - 13s 327ms/step
shape test_preds (5217,)
3-2-CW-90 has 5217 predictions with an mse of 0.07126850517879602
seq size 3
len ground_truth 1290
11/11 [==============================] - 3s 297ms/step
shape test_preds (1290,)
test90 has 1290 predictions with an mse of 0.07850961801796108
seq size 3
len ground_truth 5162
41/41 [==============================] - 14s 345ms/step
shape test_preds (5162,)
train90 has 5162 predictions with an mse of 0.06918496937407186
Getting 85-speed mse-s for fold 3
seq size 3
len ground_truth 6086
48/48 [==============================] - 16s 326ms/step
shape test_preds (6086,)
2-3-CW-85 has

INFO:donkeycar.parts.keras:input_shape (120, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model models/Keras3D_CNN_ModifiedOnlySteering-4.fold-of-5-CW.h5
INFO:donkeycar.parts.interpreter:Loading model models/Keras3D_CNN_ModifiedOnlySteering-4.fold-of-5-CW.h5


shape test_preds (4431,)
2-1-CW-80 has 4431 predictions with an mse of 0.08191703563379654
Getting 90-speed  mse-s for fold 4
seq size 3
len ground_truth 1235
10/10 [==============================] - 3s 304ms/step
shape test_preds (1235,)
2-2-CW-90 has 1235 predictions with an mse of 0.09283889687224516
seq size 3
len ground_truth 5217
41/41 [==============================] - 14s 336ms/step
shape test_preds (5217,)
3-2-CW-90 has 5217 predictions with an mse of 0.08600003309071697
seq size 3
len ground_truth 1290
11/11 [==============================] - 3s 317ms/step
shape test_preds (1290,)
test90 has 1290 predictions with an mse of 0.09133460966659902
seq size 3
len ground_truth 5162
41/41 [==============================] - 14s 335ms/step
shape test_preds (5162,)
train90 has 5162 predictions with an mse of 0.0863030927306781
Getting 85-speed mse-s for fold 4
seq size 3
len ground_truth 6086
48/48 [==============================] - 16s 335ms/step
shape test_preds (6086,)
2-3-CW-85 has 

INFO:donkeycar.parts.keras:input_shape (120, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model models/Keras3D_CNN_ModifiedOnlySteering-5.fold-of-5-CW.h5
INFO:donkeycar.parts.interpreter:Loading model models/Keras3D_CNN_ModifiedOnlySteering-5.fold-of-5-CW.h5


shape test_preds (4431,)
2-1-CW-80 has 4431 predictions with an mse of 0.08625612599684027
Getting 90-speed  mse-s for fold 5
seq size 3
len ground_truth 1235
10/10 [==============================] - 3s 289ms/step
shape test_preds (1235,)
2-2-CW-90 has 1235 predictions with an mse of 0.08036055739881535
seq size 3
len ground_truth 5217
41/41 [==============================] - 14s 331ms/step
shape test_preds (5217,)
3-2-CW-90 has 5217 predictions with an mse of 0.08027144712810623
seq size 3
len ground_truth 1290
11/11 [==============================] - 3s 304ms/step
shape test_preds (1290,)
test90 has 1290 predictions with an mse of 0.09394720011678827
seq size 3
len ground_truth 5162
41/41 [==============================] - 13s 321ms/step
shape test_preds (5162,)
train90 has 5162 predictions with an mse of 0.07687515303409889
Getting 85-speed mse-s for fold 5
seq size 3
len ground_truth 6086
48/48 [==============================] - 15s 309ms/step
shape test_preds (6086,)
2-3-CW-85 has

In [ ]:

results_unpacked = []

for fold, fold_results in CC_ModelResults.results.items():
    for speed_name, results in fold_results.items():
        for r in results:
            for pred, gt in zip(r.predictions, r.ground_truths):
                results_unpacked.append([r.name, r.direction, r.speed, r.fold, r.is_train, pred, gt])

for fold, fold_results in CW_ModelResults.results.items():
    for speed_name, results in fold_results.items():
        for r in results:
            for pred, gt in zip(r.predictions, r.ground_truths):
                results_unpacked.append([r.name, r.direction, r.speed, r.fold, r.is_train, pred, gt])

len(results_unpacked)

In [ ]:
import pandas as pd
df = pd.DataFrame(results_unpacked, columns=['name', 'direction', 'speed', 'fold', 'is_train', 'predicted_angle', 'ground_truth_angle'])


In [9]:
df.to_csv('3D-CNN-model-results.csv.gz', index=False)